# Automated_Parameter_Space_Exploration

Written by: Ketong Shao and Angelo D. Bonzanini

Date: October 2021

* Use the trained DNN, representing the pin-to-pin plasma setup
* Bayesian optimization for parameter space exploration

## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.framework import ops

seed=0
np.random.seed(seed) # fix random seed
tf.random.set_seed(seed)

from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.layers import Dense, Dropout, Flatten

from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load the experimental data
df = pd.read_csv('pintopindata.csv')

# calculate the discharge power
outpower = (np.array(df['Vdis'])*np.array(df['mA'])).reshape(-1,1)

# calculate the NOx production rate
outproduction = np.sum(np.array(df[['NO','NO2','HONO']]),axis=1).reshape(-1,1)*((np.array(df['slm'])+np.array(df['diluted'])).reshape(-1,1))

# build the outputs data
outputs = np.concatenate((outproduction,outpower),axis=1)
# extract the operating experimental parameters as inputs
inputs = np.array(df[['mA','O2frac','mm','slm']])

# shuffle the data, which is necessary for machine learning model training
np.random.seed(42)
index = list(range(len(inputs)))
np.random.shuffle(index)

outputs = outputs[index,:]
inputs = inputs[index,:]

In [3]:
# Scaler construction
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaler = scaler_x.fit(inputs)
X_scaler = scaler_y.fit(outputs)

In [4]:
# Load the trained DNN, which will represent the pin-to-pin setup
Pintopin_model = tf.keras.models.load_model('Pintopin_DNN_model')

## Bayesian optimization for parameter space exploration

In [5]:
# Necessary packages installation and importing
! pip install scikit-optimize
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer 

In [6]:
# Define the parameter space to explore

dim_mA = Real(low=20, high=70, prior='uniform',
                         name='mA')

dim_O2frac = Real(low=0, high=0.2095, prior='uniform',name='O2frac') 

dim_mm = Real(low=2, high=15, prior='uniform',name='mm') 

dim_slm = Real(low=0.5, high=7, prior='uniform',name='slm')


dimensions = [dim_mA,
              dim_O2frac,
              dim_mm,
              dim_slm
              ]

# initial value, from which the Bayesian optimization starts
default_parameters = [60, 0.11, 10.0, 6.0]

In [13]:
@use_named_args(dimensions=dimensions)
def inverse_loss_obj(mA,O2frac,mm,slm):
    input_x = np.array([mA,O2frac,mm,slm]).reshape(1,-1)
    input_norm = scaler_x.transform(input_x)
    output_now = scaler_y.inverse_transform(Pintopin_model.predict(input_norm)).reshape(-1)
    # the edge situations are not included in the data used in DNN training,
    # therefore, here needs some manual efforts to exclude them.
    if output_now[0] <= 0:
        a = 0.1
    else:
        a = output_now[0]

    if output_now[1] <= 0:
        b = 0.1
    else:
        b = output_now[1]

    current_v = b/a*60*24*1e-3/(14*2.5*1e-6) # necessary factors converting the unit to MJ/tN
    print('current energy per NOx (energy consumption) is {}'.format(current_v), 'MJ/tN')
    return current_v
    

In [15]:
inverse_result = gp_minimize(func=inverse_loss_obj,
                            dimensions=dimensions,
                            n_calls=150, 
                            noise= 1e-8,
                            n_jobs=-1,
                            acq_func="EI",
                            x0=default_parameters,)
                            #random_state=42)

current energy per NOx (energy consumption) is 406.31673165730075 MJ/tN
current energy per NOx (energy consumption) is 2543.3533532278884 MJ/tN
current energy per NOx (energy consumption) is 426.43226470266075 MJ/tN
current energy per NOx (energy consumption) is 1493.3146578924998 MJ/tN
current energy per NOx (energy consumption) is 715.2846796172006 MJ/tN
current energy per NOx (energy consumption) is 894.9689269065858 MJ/tN
current energy per NOx (energy consumption) is 412.7021900245122 MJ/tN
current energy per NOx (energy consumption) is 344.2275396415166 MJ/tN
current energy per NOx (energy consumption) is 677.8766768319266 MJ/tN
current energy per NOx (energy consumption) is 409.70823168754583 MJ/tN
current energy per NOx (energy consumption) is 611.3551727363042 MJ/tN
current energy per NOx (energy consumption) is 501.8700446401324 MJ/tN
current energy per NOx (energy consumption) is 405.1640331745148 MJ/tN
current energy per NOx (energy consumption) is 387.3248909200941 MJ/tN
c

current energy per NOx (energy consumption) is 295.78994853155956 MJ/tN
current energy per NOx (energy consumption) is 303.9709542478834 MJ/tN
current energy per NOx (energy consumption) is 295.7801584686552 MJ/tN
current energy per NOx (energy consumption) is 295.69229696478163 MJ/tN
current energy per NOx (energy consumption) is 296.13359698227475 MJ/tN
current energy per NOx (energy consumption) is 336.50473185947965 MJ/tN
current energy per NOx (energy consumption) is 296.4404608522143 MJ/tN
current energy per NOx (energy consumption) is 300.44151203972956 MJ/tN
current energy per NOx (energy consumption) is 295.70241272449493 MJ/tN
current energy per NOx (energy consumption) is 295.7718627793449 MJ/tN
current energy per NOx (energy consumption) is 295.71806532996044 MJ/tN
current energy per NOx (energy consumption) is 304.23358082771307 MJ/tN
current energy per NOx (energy consumption) is 295.6948450633458 MJ/tN
current energy per NOx (energy consumption) is 305.0295455115182 MJ/t

In [22]:
# Print the found best MSE
print('Minimal energy consumption found:', inverse_result.fun,'MJ/tN',)
print('-----------------------------------------------')
# Print the corresponding hyperparameter values
print('Operating parameter values:', inverse_result.x)
print('-----------------------------------------------')
print('Corresponding to:','\n',
      'Applied current:', inverse_result.x[0],'mA\n',
      'Oxygen mole fraction:', inverse_result.x[1],'%\n',
      'Pin-to-pin discharge distance:', inverse_result.x[2],'mm\n',
      'Flowrate:', inverse_result.x[3],'slm\n',)

Minimal energy consumption found: 295.69229696478163 MJ/tN
-----------------------------------------------
Operating parameter values: [40.505725580398874, 0.2095, 12.883727185746547, 7.0]
-----------------------------------------------
Corresponding to: 
 Applied current: 40.505725580398874 mA
 Oxygen mole fraction: 0.2095 %
 Pin-to-pin discharge distance: 12.883727185746547 mm
 Flowrate: 7.0 slm

